In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib
import sklearn
import keras
import sys

from pandas.plotting import scatter_matrix
from sklearn.metrics import accuracy_score
from keras.models import Sequential
from sklearn import model_selection
from keras.layers import Dense
from keras import regularizers
from keras import optimizers

In [ ]:
data = pd.read_csv('dados.csv', sep=',')
X = np.array(data.drop(['MADE'], axis=1))
y = np.array(data['MADE'])

## Rede Neural

### Criando a rede neural

In [ ]:
from keras.callbacks import EarlyStopping

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, random_state=2, test_size=0.2)

# Criando a arquitetura da rede neural
model = Sequential()

# Rede configurada a partir de informações da dimensão VC e da regra de ouro
#model.add(Dense(5, input_dim=12, kernel_initializer='normal',  activation='tanh'))
model.add(Dense(5, input_dim=12, kernel_initializer='normal', kernel_regularizer=regularizers.l2(0.01), activation='relu'))
model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))

# Early stopping
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=20)

# Instanciação e configuração de otimizadores
sgd = optimizers.SGD(learning_rate=0.01)

# Compila o modelo. O otimizador escolhido foi o Adam com os parâmetros default
model.compile(loss='binary_crossentropy', optimizer="adam", metrics=['accuracy'])

print(model.summary())

### Treinando a rede neural

In [ ]:
# 167700 exemplos no 'X_train', para calcular a quantidade mínima de neurônios com base na regra de ouro:
# n <= ( (167700 - 10) / (10*(12+2) ) )  --> n <= 1197

In [ ]:
BATCH_SIZE = 32

X_train, X_val, y_train, y_val = model_selection.train_test_split(X_train, y_train, random_state=2, test_size=0.2)

# Treinamento SEM Early Stopping
#history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=BATCH_SIZE)

# Treinamento COM Early Stopping
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=BATCH_SIZE, callbacks=[es])


### Performance do modelo

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Metrica de erro')
plt.ylabel('Erro')
plt.xlabel('Epoca')
plt.legend(['Treinamento', 'Validacao'])
plt.show()

pred = np.round(model.predict(X_test))
pred_train = np.round(model.predict(X_train))

print(f'--> Acuracia (train): {accuracy_score(y_train, pred_train):.4f}')
print(f'--> Acuracia (test): {accuracy_score(y_test, pred):.4f}')